<a href="https://colab.research.google.com/github/datacamp/data-analysis-in-sql-live-training/blob/master/notebooks/PostgreSQL_live_session_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/datacamp/data-analysis-in-sql-live-session/blob/master/assets/datacamp.svg?raw=True" alt = "DataCamp icon" width="50%">
</p>
<br><br>

## **Data Analysis with SQL**

In this webinar, you'll learn how to write advanced queries to calculate core business metrics and KPIs. You'll be able to:

* Use Common Table Expressions to temporarily store a query's results
* Fetch values from different rows using window functinos
* Use self-joins to peak into the future

## **The Dataset**


We'll use one table, `user_sessions`, which stores data user session data on a social media website. The table's schema is as follows:

- `session_date`: The date on which the user accessed the site
- `user_id`: The user's unique identifier
- `time_spent_in_mins`: How much time the user spent on the site


## **Setting up PostgreSQL**

In [0]:
#@title **This block of code will install PosgreSQL**
%%capture
!wget -qO- https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ bionic-pgdg main" >/etc/apt/sources.list.d/pgdg.list
!apt -qq update
!apt -yq install postgresql-12 postgresql-client-12
!service postgresql start
# make calling psql shorter
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"  
!psql postgres -c "CREATE DATABASE root"  # now just !psql -c "..."
# load SQL extensions
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

In [0]:
#@title **This will download your data to local environment**
!wget -q https://github.com/datacamp/data-analysis-in-sql-live-training/raw/master/data/user_data.csv

In [4]:
#@title **This will create your table**
%%sql
-- Make sure to amend you table name, column names and types
DROP TABLE IF EXISTS user_sessions;
CREATE TABLE user_sessions(
 session_date date,
 user_id int,
 time_spent_in_mins int
);

COPY user_sessions
-- Make sure to point to correct file and delimiter 
FROM '/content/user_data.csv' DELIMITER ',' CSV HEADER;

 * postgresql+psycopg2://@/postgres


""


Let's start by exploring the table.

In [5]:
%%sql

-- SELECT first 5 rows

SELECT *
FROM user_sessions 
LIMIT 5;

 * postgresql+psycopg2://@/postgres


,session_date,user_id,time_spent_in_mins
0,2020-01-01,1,127
1,2020-01-01,2,147
2,2020-01-01,5,106
3,2020-01-01,6,179
4,2020-01-01,7,143


## Active users

The active users KPI counts the active users of a company's app over a certain time period:
- by day (daily active users, or DAU)
- by month (monthly active users, or MAU)

For example, Facebook had 1.76B DAU and 2.6 MAU in March.

Stickiness (DAU / MAU) measures how often users engage with an app on average. Facebook's stickiness for March was `1.76B / 2.6B ~= 0.677`, meaning that, on average, users used Facebook for `67.7% x 30 days ~= 20` days each month.

In [6]:
%%sql

-- Calculate the Daily Active Users (DAU)

SELECT DISTINCT
  session_date,
  COUNT(DISTINCT user_id) AS users
FROM user_sessions
GROUP BY session_date
ORDER BY session_date ASC;

 * postgresql+psycopg2://@/postgres


,session_date,users
0,2020-01-01,777
1,2020-01-02,755
2,2020-01-03,786
3,2020-01-04,760
4,2020-01-05,749
...,...,...
147,2020-05-27,1034
148,2020-05-28,1066
149,2020-05-29,1023
150,2020-05-30,1050


## Monthly active users

Usually, reports include MAU, not DAU. How do you convert the session dates to months?

### Enter `DATE_TRUNC`

`DATE_TRUNC(date_part, date) → DATE`: Truncates `date` to the nearest `date_part`.

#### Examples
- `DATE_TRUNC('week', '2018-06-12') :: DATE → '2018-06-11'`
- `DATE_TRUNC('month', '2018-06-12') :: DATE → '2018-06-01'`
- `DATE_TRUNC('quarter', '2018-06-12') :: DATE → '2018-04-01'`
- `DATE_TRUNC('year', '2018-06-12') :: DATE → '2018-01-01'`

**Note**: `:: DATE` is just to remove the hours, minutes, and seconds.

In [7]:
%%sql

-- Calculate the Monthly Active Users (DAU)

SELECT
  DATE_TRUNC('month', session_date) :: DATE AS session_month,
  COUNT(DISTINCT user_id) AS active_users
FROM user_sessions
GROUP BY session_month
ORDER BY session_month ASC;

 * postgresql+psycopg2://@/postgres


,session_month,active_users
0,2020-01-01,1472
1,2020-02-01,1621
2,2020-03-01,1723
3,2020-04-01,1879
4,2020-05-01,1995


## Registered users

What if you want to use a query's results in another query? Let's define the user's registration date as the date of that user's first session.

Your goal here is to calculate the growth in registrations; the growth rate of new users is another important KPI for platforms.

In [8]:
%%sql

-- Get each user's registration date

SELECT
  user_id,
  MIN(session_date) AS reg_date
FROM user_sessions
GROUP BY user_id
ORDER BY reg_date ASC, user_id ASC;

 * postgresql+psycopg2://@/postgres


,user_id,reg_date
0,1,2020-01-01
1,2,2020-01-01
2,5,2020-01-01
3,6,2020-01-01
4,7,2020-01-01
...,...,...
2056,1953,2020-05-11
2057,2051,2020-05-11
2058,1969,2020-05-13
2059,2040,2020-05-15


### Common Table Expressions (CTEs)

Now that you have each user's registration date, you'll want to store the results somehow to use them in a different query. How do you do that?

```sql
WITH cte_name AS (
  ...
)

SELECT *
FROM cte_name;
```

A CTE stores the results of a query temporarily in the specificed `cte_name` so it can be used in the outer query later on.

In [10]:
%%sql

-- Store each user's registration date in the regs CTE
-- Calculate the number of registrations per month

WITH regs AS (
  SELECT
    user_id,
    MIN(session_date) AS reg_date
  FROM user_sessions
  GROUP BY user_id)

SELECT
  DATE_TRUNC('month', reg_date) :: DATE AS reg_month,
  COUNT(DISTINCT user_id) AS users
FROM regs
GROUP BY reg_month
ORDER BY reg_month ASC;

 * postgresql+psycopg2://@/postgres


,reg_month,users
0,2020-01-01,1472
1,2020-02-01,166
2,2020-03-01,147
3,2020-04-01,152
4,2020-05-01,124


## Growth and window functions

You now have each month's registrations. How do you calculate growth?

`Growth = (Current month - previous month) / previous month`

For example, if you had 122 registrations last month, and you have 156 registrations this month, your registrations grew by `(156 - 122) / 122 ~= 28%` this month.

So you need both the previous and the current months' registrations in the same row. How do you do that?

### Window functions

A window function performs some operation across a set of table rows that are somehow related to the current row.

- `LAG(column_a, 1) OVER (ORDER BY column_b ASC)` Gets the previous row's value in `column_a` if you sort by `column_b`.

In [14]:
%%sql
-- Fetch the previous and current months' MAUs


WITH regs AS (
  SELECT
    user_id,
    MIN(session_date) AS reg_date
  FROM user_sessions
  GROUP BY user_id),

  regs_per_month AS (
    SELECT
      DATE_TRUNC('month', reg_date) :: DATE AS reg_month,
      COUNT(DISTINCT user_id) AS users
    FROM regs
    GROUP BY reg_month
  )

SELECT
  reg_month,
  users,
  LAG(users, 1) OVER (ORDER BY regs_per_month ASC) AS previous_users
FROM regs_per_month
ORDER BY reg_month ASC;

 * postgresql+psycopg2://@/postgres


,reg_month,users,previous_users
0,2020-01-01,1472,NaN
1,2020-02-01,166,1472.0
2,2020-03-01,147,166.0
3,2020-04-01,152,147.0
4,2020-05-01,124,152.0


Store the results in a CTE and apply the formula to get the monthly registrations growth rates. You can use `COALESCE(..., 1)` to convert any `NULL` values to 1.

In [18]:
%%sql

-- Calculate the monthly growth in registrations

WITH regs AS (
  SELECT
    user_id,
    MIN(session_date) AS reg_date
  FROM user_sessions
  GROUP BY user_id),

  regs_per_month AS (
    SELECT
      DATE_TRUNC('month', reg_date) :: DATE AS reg_month,
      COUNT(DISTINCT user_id) AS users
    FROM regs
    GROUP BY reg_month
  ),

  lag_regs AS (
    SELECT
      reg_month,
      users,
      COALESCE(
        LAG(users, 1) OVER (ORDER BY regs_per_month ASC), 1
      ) AS previous_users
    FROM regs_per_month)

SELECT
  reg_month,
  users,
  ROUND(
      (users - previous_users) :: NUMERIC / previous_users, 3
  ) AS growth_rate
FROM lag_regs
ORDER BY reg_month ASC;

 * postgresql+psycopg2://@/postgres


,reg_month,users,growth_rate
0,2020-01-01,1472,1471.000
1,2020-02-01,166,-0.887
2,2020-03-01,147,-0.114
3,2020-04-01,152,0.034
4,2020-05-01,124,-0.184


## Retained and resurrected users

Users can be split into four groups:
- New/registered users are ones that just signed up for your platform
- Retained users used to use your app, and still do, too.
- Churned users used to use your app, and no longer do.
- Resurrected users were churned users who returned to using your app.

Retention is another core KPI that platforms use to measure how well they are at keeping their users.

In [19]:
%%sql

-- Get the months in which each user is active

SELECT DISTINCT
  DATE_TRUNC('month', session_date) :: DATE AS act_month,
  user_id
FROM user_sessions
ORDER BY act_month ASC, user_id ASC;

 * postgresql+psycopg2://@/postgres


,act_month,user_id
0,2020-01-01,0
1,2020-01-01,1
2,2020-01-01,2
3,2020-01-01,3
4,2020-01-01,4
...,...,...
8685,2020-05-01,2056
8686,2020-05-01,2057
8687,2020-05-01,2058
8688,2020-05-01,2059


## Self-joins

Now that you have the months in which each user is active, how do you calculate retention?

If you left-join this table on itself on the same user ID and having a one-month difference in users, you'll see whether a user is still active in the next month or not. If the user isn't active, then the user is churned. The count of non-`NULL`s is the count of retained users.

```sql
...
FROM ... AS prev
LEFT JOIN ... AS curr
  ON prev.user_id = curr.user_id
 AND prev.month = (curr.month - INTERVAL '1 MONTH')
...
```

In [28]:
%%sql

-- Get whether each user churned in a given month

WITH act_months AS (
  SELECT DISTINCT
    DATE_TRUNC('month', session_date) :: DATE AS act_month,
    user_id
  FROM user_sessions)

SELECT
  prev.act_month,
  prev.user_id,
  curr.user_id IS NULL AS churned_next_month
FROM act_months AS prev
LEFT JOIN act_months AS curr
  ON prev.user_id = curr.user_id
 AND prev.act_month = (curr.act_month - INTERVAL '1 MONTH')
ORDER BY prev.act_month ASC, prev.user_id ASC;

 * postgresql+psycopg2://@/postgres


,act_month,user_id,churned_next_month
0,2020-01-01,0,False
1,2020-01-01,1,False
2,2020-01-01,2,False
3,2020-01-01,3,False
4,2020-01-01,4,False
...,...,...,...
8685,2020-05-01,2056,True
8686,2020-05-01,2057,True
8687,2020-05-01,2058,True
8688,2020-05-01,2059,True


Store the results in a CTE and count the number of `FALSE` in the `churned_next_month` to get the retention rate.

In [34]:
%%sql

-- Get whether each user churned in a given month

WITH act_months AS (
    SELECT DISTINCT
      DATE_TRUNC('month', session_date) :: DATE AS act_month,
      user_id
    FROM user_sessions),

  churned AS (
    SELECT
      prev.act_month,
      prev.user_id,
      curr.user_id IS NULL AS churned_next_month
    FROM act_months AS prev
    LEFT JOIN act_months AS curr
      ON prev.user_id = curr.user_id
    AND prev.act_month = (curr.act_month - INTERVAL '1 MONTH'))
  
SELECT
  act_month,
  COUNT(DISTINCT user_id) AS users,
  SUM(CASE WHEN NOT churned_next_month THEN 1 ELSE 0 END) AS retained_next_month,
  ROUND(
      SUM(CASE WHEN NOT churned_next_month THEN 1 ELSE 0 END) :: NUMERIC /
      COUNT(DISTINCT user_id), 3
  ) AS retention_rate
FROM churned
GROUP BY act_month
ORDER BY act_month ASC;

 * postgresql+psycopg2://@/postgres


,act_month,users,retained_next_month,retention_rate
0,2020-01-01,1472,1455,0.988
1,2020-02-01,1621,1559,0.962
2,2020-03-01,1723,1667,0.967
3,2020-04-01,1879,1816,0.966
4,2020-05-01,1995,0,0.000
